In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

Add Train and Test CSV Datasets

In [20]:
from google.colab import files
uploaded = files.upload()

Saving heart_test.csv to heart_test.csv
Saving heart_train.csv to heart_train.csv


In [0]:
#Uncomment the following code to remove files (for reupload purpose)
!rm "heart_train.csv"
!rm "heart_test.csv"


Get data from CSV Files

In [0]:
#Followed tutorial from https://www.tensorflow.org/tutorials/load_data/csv
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [0]:
LABEL_COLUMN = 'chd'
LABELS = [0, 1]

def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [41]:
raw_train_data = get_dataset('heart_train.csv')
raw_test_data = get_dataset('heart_test.csv')

show_batch(raw_train_data)

sbp                 : [146 136 122 170 162]
tobacco             : [10.5   1.7   0.72  7.6   6.94]
ldl                 : [8.29 3.53 4.04 5.5  4.55]
adiposity           : [35.36 20.13 32.38 37.83 33.36]
famhist             : [b'Present' b'Absent' b'Absent' b'Present' b'Present']
typea               : [78 56 34 42 52]
obesity             : [32.73 19.44 28.34 37.41 27.09]
alcohol             : [13.89 14.4   0.    6.17 32.06]
age                 : [53 55 55 54 43]


In [45]:
example_batch, labels_batch = next(iter(raw_train_data))

class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

NUMERIC_FEATURES = ['sbp','tobacco','ldl', 'adiposity', 'typea', 'obesity', 'alcohol', 'age']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

show_batch(packed_train_data)

example_batch, labels_batch = next(iter(packed_train_data))

famhist             : [b'Absent' b'Absent' b'Absent' b'Present' b'Present']
numeric             : [[118.     0.12   1.96  20.31  37.    20.01   2.42  18.  ]
 [143.     0.46   2.4   22.87  62.    29.17  15.43  29.  ]
 [122.     0.28   4.19  19.97  61.    25.63   0.    24.  ]
 [130.     0.54   3.63  22.03  69.    24.34  12.86  39.  ]
 [134.    14.1    4.44  22.39  65.    23.09   0.    40.  ]]


In [0]:
numeric_column = tf.feature_column.numeric_column('numeric', shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]

numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)

CATEGORIES = {
    'famhist': ['Absent', 'Present']
}

categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)


In [67]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

print("--Fit model--")
model.fit(train_data, epochs=500)

print("--Evaluate Train model--")
model_loss, model_acc = model.evaluate(train_data, verbose=2)
print(f"Model Loss:    {model_loss:.2f}")
print(f"Model Accuracy: {model_acc*100:.1f}%")

print("--Evaluate Test model--")
model_loss, model_acc = model.evaluate(test_data, verbose=2)
print(f"Model Loss:    {model_loss:.2f}")
print(f"Model Accuracy: {model_acc*100:.1f}%")

--Fit model--
Epoch 1/500
83/83 [==============================] - 1s 11ms/step - loss: 1.4349 - accuracy: 0.5850
Epoch 2/500
83/83 [==============================] - 0s 3ms/step - loss: 0.8767 - accuracy: 0.6214
Epoch 3/500
83/83 [==============================] - 0s 3ms/step - loss: 1.0095 - accuracy: 0.6165
Epoch 4/500
83/83 [==============================] - 0s 3ms/step - loss: 0.6894 - accuracy: 0.6359
Epoch 5/500
83/83 [==============================] - 0s 3ms/step - loss: 0.6310 - accuracy: 0.6383
Epoch 6/500
83/83 [==============================] - 0s 3ms/step - loss: 0.6520 - accuracy: 0.6359
Epoch 7/500
83/83 [==============================] - 0s 3ms/step - loss: 0.6769 - accuracy: 0.6456
Epoch 8/500
83/83 [==============================] - 0s 3ms/step - loss: 0.6132 - accuracy: 0.6602
Epoch 9/500
83/83 [==============================] - 0s 3ms/step - loss: 0.6559 - accuracy: 0.6456
Epoch 10/500
83/83 [==============================] - 0s 3ms/step - loss: 0.6308 - accuracy: 0